In [ ]:
from path_arquivos import *
import pandas as pd
import numpy as np
import glob
import os
base = pd.read_excel(ar_xlsx.ar_96, usecols=['CODPROD', 'DESCRICAO'])

c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df = base.loc[base['RUA'].between(1,39)].copy()
dic_raname = {
    'ABASTECEPALETE' : 'FLEG_ABST',
    'CAPACIDADE' : 'CAP'
}
df = df.rename(columns=dic_raname)
concat = df['RUA'].astype(str) + " - " + df['PREDIO'].astype(str)

df['extrator'] = df['EMBALAGEMMASTER'].str.extract(r'(\d+)').astype(int)
df['volume_master'] = df['ALTURAARM'].astype(float) * df['LARGURAARM'].astype(float) * df['COMPRIMENTOARM'].astype(float) 
df['volume_venda'] = (df['ALTURAM3'].astype(float) * df['LARGURAM3'].astype(float) * df['COMPRIMENTOM3'].astype(float)) * df['extrator']
df['CONT_AP'] = concat.map(concat.value_counts())
df['STATUS_PROD'] = np.where(df['CONT_AP'] > 3, "DIV", 
    np.where(df['CONT_AP'] > 2,"VAL", "INT"))

df["VAR_CAP"] = np.where((df['CAP'] > df['QTTOTPAL']) & (df['STATUS_PROD'] != "INT"), "DIVERGENCIA", "NORMAL")
df['VAR_ABST'] =    np.where((df['FLEG_ABST'] == 'SIM') & (df['STATUS_PROD'] == "INT"), "NORMAL",
                    np.where((df['FLEG_ABST']== 'NÃO') & (df['STATUS_PROD'] == "DIV"), "NORMAL",
                    "DIVERGENCIA"))

df['VAR_CUBAGEM'] = np.where(df['volume_master'] < df['volume_venda'], "DIVERGENCIA", "NORMAL")

drop_col = ['PKESTRU', 'PK_END', 'CARACTERISTICA', 'PULMAO','TIPO_1', 'PRAZOVAL', 'PERCTOLERANCIAVAL', 'DTULTENT','USAWMS', 'REVENDA', 'TIPOPROD','CODFORNEC', 'FORNECEDOR', 'TIPO', 'PONTOREPOSICAO','CODAUXILIAR', 'EMBALAGEM', 'EMBALAGEMMASTER', 'QTUNITCX', 'CODSEC','SECAO', 'LASTROPAL', 'ALTURAPAL', 'QTTOTPAL', 'ALTURAM3', 'LARGURAM3','COMPRIMENTOM3', 'ALTURAARM', 'LARGURAARM', 'COMPRIMENTOARM','PESOBRUTOMASTER', 'PESOLIQMASTER', 'PESOBRUTO', 'PESOLIQ','OBS2', 'CODAUXILIAR2','volume_venda', 'volume_master','CODFILIAL']
df = df.drop(columns=drop_col)

In [86]:

var_cap = df.groupby("VAR_CAP").agg(
            TT_PROD = ('CODPROD', 'nunique')
        ).reset_index()

var_abst = df.groupby("VAR_ABST").agg(
            TT_PROD = ('CODPROD', 'nunique')
        ).reset_index()

var_cubagem = df.groupby("VAR_CUBAGEM").agg(
            TT_PROD = ('CODPROD', 'nunique')
        ).reset_index()
display(df.columns)
display(var_cubagem)


Index(['CODFILIAL', 'CODPROD', 'DESCRICAO', 'CAP', 'FLEG_ABST', 'RUA',
       'PREDIO', 'NIVEL', 'APTO', 'extrator', 'CONT_AP', 'STATUS_PROD',
       'VAR_CAP', 'VAR_ABST', 'VAR_CUBAGEM'],
      dtype='object')

,VAR_CUBAGEM,TT_PROD
0,DIVERGENCIA,4320
1,NORMAL,5641
